In [3]:
%load_ext autoreload
%autoreload 2
import os 
import spikeinterface as si

# set project path
proj_path = "/gpfs/bbp.cscs.ch/project/proj85/home/laquitai/preprint_2023/"
os.chdir(proj_path)

from src.nodes import utils
from src.nodes.utils import get_config
from src.nodes.analysis.failures import accuracy as acc
from src.nodes.metrics import quality
from src.nodes.metrics.quality import get_scores_for_dense_probe as gscdp
from src.nodes.metrics.quality import get_chance_for_dense_probe as gchdp
from src.nodes.metrics.quality import combine_quality_across_dense_probe as cqadb

# DATASETS

# NPX PROBE
# biophy spont (10m)
cfg_ns, _ = get_config("silico_neuropixels", "concatenated").values()
REC_ns = cfg_ns["probe_wiring"]["full"]["output"]

2024-09-01 13:54:46,993 - root - utils.py - get_config - INFO - Reading experiment config.
2024-09-01 13:54:47,084 - root - utils.py - get_config - INFO - Reading experiment config. - done


### Sample every second samples

In [2]:
Wired = si.load_extractor(REC_ns)
from spikeinterface import NumpyRecording


def sample_2X_traces(Wired):

    # downsample traces
    traces = Wired.get_traces()
    new_traces = traces[::2, :]

    # copy parameters
    sf = Wired.get_sampling_frequency()
    cid = Wired.get_channel_ids()

    # create new RecordingExtractor
    Wired_new = NumpyRecording([new_traces], sampling_frequency=sf, channel_ids=cid)

    # copy annotations
    for key in Wired.get_annotation_keys():
        if not key == "is_filtered":
            Wired_new.set_annotation(key, Wired.get_annotation(key))

    # copy properties
    for key in Wired.get_property_keys():
        Wired_new.set_property(key, Wired.get_property(key))

    # copy probe
    if Wired.has_probe():
        probe = Wired.get_probe()
        Wired_new.set_probe(probe)
    return Wired_new

In [3]:
Wired2X = sample_2X_traces(Wired)